In [42]:
import sys
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import time
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import f1_score,accuracy_score,roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
import os
import matplotlib.pyplot as plt
import matplotlib.pylab as pl
#import matplotlib
# matplotlib.use('Agg')
import lightgbm as lgb
from sklearn import metrics
from numpy.ma import MaskedArray
import sklearn.utils.fixes
sklearn.utils.fixes.MaskedArray = MaskedArray
from sklearn.model_selection import train_test_split
#from sklearn.metrics import plot_confusion_matrix
import shap

In [2]:
# **************** Loading the data
# For the example I am usign a table in csv format, but if your data are larger you can use other formats compatible with pandas, like .pickle
print("Start loading data file ")

cols_names=['area', 'perimeter', 'neighbours', 'max neighbour distance',
       'min neighbour distance', 'max vertices distance',
       'min vertices distance', 'max vertices-point distance',
       'min vertices-point distance', 'distance to center', 'activity',
       'particle type']
input_file='particles-features-0.1-Fa60.txt'
data = pd.read_csv(input_file, delimiter=' ',names=cols_names)

#print(data)


# *** Split the available data in training and test
# the model will be trained only using the training data
# so we can evaluate the performance on a different (test) set, that is new for the ML model
df_train, df_test = train_test_split(data, random_state=50, test_size=0.3)

print('------------ BEGIN TRAIN DATAFRAME COLUMNS ------------------')
print(df_train.columns)
print('------------- END TRAIN DATAFRAME COLUMNS -------------------')


Start loading data file 
------------ BEGIN TRAIN DATAFRAME COLUMNS ------------------
Index(['area', 'perimeter', 'neighbours', 'max neighbour distance',
       'min neighbour distance', 'max vertices distance',
       'min vertices distance', 'max vertices-point distance',
       'min vertices-point distance', 'distance to center', 'activity',
       'particle type'],
      dtype='object')
------------- END TRAIN DATAFRAME COLUMNS -------------------


In [3]:
df_train

,area,perimeter,neighbours,max neighbour distance,min neighbour distance,max vertices distance,min vertices distance,max vertices-point distance,min vertices-point distance,distance to center,activity,particle type
635067,0.767525,3.308608,6.0,1.153157,0.784682,1.128429,0.353839,0.638007,0.507887,0.071155,1.0,0.0
1849450,0.789758,3.367023,7.0,1.158468,0.840325,1.130436,0.115149,0.588227,0.543554,0.021318,0.0,1.0
1512226,0.862553,3.492780,7.0,1.278831,0.906190,1.263235,0.118974,0.652097,0.517653,0.017999,0.0,1.0
904884,0.741425,3.316321,5.0,1.017501,0.834547,1.164606,0.544636,0.655106,0.524760,0.031100,0.0,0.0
161732,0.754256,3.268442,6.0,1.028061,0.810904,1.148971,0.311192,0.609138,0.517849,0.013343,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
441633,0.870017,3.553290,6.0,1.195560,0.806754,1.306405,0.387368,0.706019,0.540799,0.034360,0.0,1.0
677997,0.926657,3.668523,6.0,1.273297,0.861496,1.303061,0.154767,0.672960,0.568200,0.022512,0.0,0.0
239499,1.072487,3.900515,7.0,1.391715,0.965872,1.313825,0.141951,0.722786,0.571711,0.049071,0.0,1.0
1152480,0.881689,3.535999,6.0,1.136420,0.872215,1.266622,0.444050,0.636505,0.551523,0.033578,0.0,1.0


In [4]:
df_test

,area,perimeter,neighbours,max neighbour distance,min neighbour distance,max vertices distance,min vertices distance,max vertices-point distance,min vertices-point distance,distance to center,activity,particle type
614099,0.791039,3.335474,7.0,1.226071,0.846609,1.133161,0.168695,0.640357,0.497096,0.065427,1.0,0.0
903904,0.854923,3.480301,7.0,1.291994,0.874127,1.190559,0.080361,0.654602,0.514242,0.043921,0.0,0.0
23774,0.767726,3.361134,5.0,1.058551,0.847403,1.172293,0.518810,0.602921,0.538458,0.024555,0.0,0.0
1727994,0.705179,3.206959,5.0,0.929480,0.845224,1.069255,0.562249,0.558434,0.533297,0.006369,0.0,0.0
1714002,0.988856,3.737136,6.0,1.296641,0.948468,1.279418,0.376083,0.684682,0.562662,0.051151,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1540347,0.813158,3.444021,6.0,1.126402,0.850171,1.160740,0.156795,0.629831,0.567769,0.044143,0.0,1.0
900741,0.753771,3.320463,5.0,0.964334,0.854351,1.112026,0.604748,0.598057,0.533787,0.018898,0.0,0.0
358526,0.983468,3.785026,6.0,1.362060,0.947161,1.375282,0.178270,0.704584,0.545055,0.032643,0.0,1.0
1491630,0.837051,3.440542,6.0,1.066545,0.840857,1.177933,0.326667,0.643324,0.536564,0.043843,0.0,1.0


In [3]:
print("create x/y dataframes (train set)")
# In this particulat proble we want to identify if a specific particle 'is_active' or not
# so we have to do a binary classification of the column 'is_active'
df_train_y = df_train['activity'].copy().astype('int')
df_train.drop(columns='activity', inplace=True)
df_train_x = df_train

print("create x/y dataframes (test set)")
df_test_y = df_test['activity'].copy().astype('int')
df_test.drop(columns='activity', inplace=True)
df_test_x = df_test

create x/y dataframes (train set)
create x/y dataframes (test set)


In [4]:
print('\n\n*** Using a gradient boosting algorithm')
# This is a more advanced algorithm that is explained here:https://lightgbm.readthedocs.io/en/latest/pythonapi/lightgbm.LGBMClassifier.html
model = lgb.LGBMClassifier(learning_rate=0.01,num_leaves=100,max_depth=15, early_stopping_rounds=10, num_iterations=3000, random_state=42)

model.fit(df_train_x,df_train_y,eval_set=[(df_train_x,df_train_y),(df_test_x,df_test_y)],
            eval_metric='binary')
print('\n\n***Training accuracy {:.4f}'.format(model.score(df_train_x,df_train_y)))
print('***Testing accuracy {:.4f}\n'.format(model.score(df_test_x,df_test_y)))

print(metrics.classification_report(df_test_y,model.predict(df_test_x)))



*** Using a gradient boosting algorithm


c:\Users\Pedro\Documentos\pruebatfg\myenvprueba\lib\site-packages\lightgbm\engine.py:204: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] early_stopping_round is set=10, early_stopping_rounds=10 will be ignored. Current value: early_stopping_round=10
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 140128, number of negative: 1259872
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.029029 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2306
[LightGBM] [Info] Number of data points in the train set: 1400000, number of used features: 11
[LightGBM] [Warning] early_stopping_round is set=10, early_stopping_rounds=10 will be ignored. Current value: early_stopping_round=10
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.100091 -> initscore=-2.196209
[LightGBM] [Info] Start training from score -2.196209
Training until validation scores don't improve for 10 rounds
Early stoppi

In [5]:
pred_test=model.predict(df_test_x)

In [6]:
f1_score(df_test_y,pred_test)

0.24437836578648067

In [23]:
#Returns accuracy,auc,f1_score
def score(y,pred):
    return accuracy_score(y,pred),roc_auc_score(y,pred),f1_score(y,pred)

In [22]:
df=pd.read_csv("phia0.1/traj_phia0.1-T05-Fa20-tau1.dat")
df 


,1 1 7.32901 24.9911
0,2 1 24.2163 28.0885
1,3 1 24.8592 2.49579
2,4 1 16.2195 8.41785
3,5 1 10.2535 24.3277
4,6 1 19.2187 23.6963
...,...
2000994,996 4 26.0632 4.15549
2000995,997 4 23.9477 22.5588
2000996,998 4 25.0032 23.802
2000997,999 4 16.861 17.2506


In [26]:
def gb_model(df_train_x,df_train_y,df_test_x,df_test_y):
    model = lgb.LGBMClassifier(learning_rate=0.01,num_leaves=100,max_depth=15, early_stopping_rounds=10, num_iterations=3000, random_state=42)

    model.fit(df_train_x,df_train_y,eval_set=[(df_train_x,df_train_y),(df_test_x,df_test_y)],
            eval_metric='binary')
    return model

In [30]:
def plot_score_Fa_depend(fa_list,density,model_method):
    cols_names=['area', 'perimeter', 'neighbours', 'max neighbour distance',
       'min neighbour distance', 'max vertices distance',
       'min vertices distance', 'max vertices-point distance',
       'min vertices-point distance', 'distance to center', 'activity',
       'particle type']
    accuracy_list=[]
    auc_list=[]
    f1_list=[]

    for fa in fa_list:
        input_file=f"phia{density}/particles-features-{density}-Fa{fa}.txt"
        data = pd.read_csv(input_file, delimiter=' ',names=cols_names)

        df_train, df_test = train_test_split(data, random_state=50, test_size=0.3)
        df_train_y = df_train['activity'].copy().astype('int')
        df_train.drop(columns='activity', inplace=True)
        df_train_x = df_train

        df_test_y = df_test['activity'].copy().astype('int')
        df_test.drop(columns='activity', inplace=True)
        df_test_x = df_test

        model=model_method(df_train_x,df_train_y,df_test_x,df_test_y)
        pred=model.predict(df_test_x)
        accuracy,auc,f1=score(df_test_y,pred)
        accuracy_list.append(accuracy)
        auc_list.append(auc)
        f1_list.append(f1)
    
    plt.scatter(fa_list,accuracy_list)
    plt.scatter(fa_list,auc_list)
    plt.scatter(fa_list,f1_list)
    plt.legend(['Accuracy','ROC AUC score', 'F1 score'])
    return accuracy_list,auc_list,f1_list
        

In [31]:
Fa_list=[100,85,75,60,40,20]
plot_score_Fa_depend(Fa_list,0.1,gb_model)

c:\Users\Pedro\Documentos\pruebatfg\myenvprueba\lib\site-packages\lightgbm\engine.py:204: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] early_stopping_round is set=10, early_stopping_rounds=10 will be ignored. Current value: early_stopping_round=10
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 13991, number of negative: 126009
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000643 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2306
[LightGBM] [Info] Number of data points in the train set: 140000, number of used features: 11
[LightGBM] [Warning] early_stopping_round is set=10, early_stopping_rounds=10 will be ignored. Current value: early_stopping_round=10
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.099936 -> initscore=-2.197939
[LightGBM] [Info] Start training from score -2.197939
Training unt

c:\Users\Pedro\Documentos\pruebatfg\myenvprueba\lib\site-packages\lightgbm\engine.py:204: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] early_stopping_round is set=10, early_stopping_rounds=10 will be ignored. Current value: early_stopping_round=10
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 4234, number of negative: 37766
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000986 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2305
[LightGBM] [Info] Number of data points in the train set: 42000, number of used features: 11
[LightGBM] [Warning] early_stopping_round is set=10, early_stopping_rounds=10 will be ignored. Current value: early_stopping_round=10
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.100810 -> initscore=-2.188262
[LightGBM] [Info] Start training from score -2.188262
Training until validation scores don't improve for 10 rounds
Early stopping, be

c:\Users\Pedro\Documentos\pruebatfg\myenvprueba\lib\site-packages\lightgbm\engine.py:204: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


...

c:\Users\Pedro\Documentos\pruebatfg\myenvprueba\lib\site-packages\lightgbm\engine.py:204: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] early_stopping_round is set=10, early_stopping_rounds=10 will be ignored. Current value: early_stopping_round=10
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 140097, number of negative: 1259903
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.030301 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2305
[LightGBM] [Info] Number of data points in the train set: 1400000, number of used features: 11
[LightGBM] [Warning] early_stopping_round is set=10, early_stopping_rounds=10 will be ignored. Current value: early_stopping_round=10
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.100069 -> initscore=-2.196455
[LightGBM] [Info] Start training from score -2.196455
Training until validation scores don't improve for 10 rounds
Early stoppi

c:\Users\Pedro\Documentos\pruebatfg\myenvprueba\lib\site-packages\lightgbm\engine.py:204: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] early_stopping_round is set=10, early_stopping_rounds=10 will be ignored. Current value: early_stopping_round=10
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 140115, number of negative: 1259885
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.028570 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2304
[LightGBM] [Info] Number of data points in the train set: 1400000, number of used features: 11
[LightGBM] [Warning] early_stopping_round is set=10, early_stopping_rounds=10 will be ignored. Current value: early_stopping_round=10
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.100082 -> initscore=-2.196312
[LightGBM] [Info] Start training from score -2.196312
Training until validation scores don't improve for 10 rounds
Early stoppi

([0.91495,
  0.9122777777777777,
  0.9111533333333334,
  0.908095,
  0.9034833333333333,
  0.900395],
 [0.6266326880740732,
  0.5897841254551394,
  0.5950115028085902,
  0.5705903378676136,
  0.5340524729333177,
  0.5031788887870942],
 [0.3852547885796892,
  0.29665924276169264,
  0.3100538413750259,
  0.24437836578648067,
  0.1303499023877459,
  0.01363283764379673])

In [40]:
fa_list=[100,85,75,60,40,20]
accuracy_list=[0.91495,
  0.9122777777777777,
  0.9111533333333334,
  0.908095,
  0.9034833333333333,
  0.900395]
auc_list=[0.6266326880740732,
  0.5897841254551394,
  0.5950115028085902,
  0.5705903378676136,
  0.5340524729333177,
  0.5031788887870942]
f1_list=[0.3852547885796892,
  0.29665924276169264,
  0.3100538413750259,
  0.24437836578648067,
  0.1303499023877459,
  0.01363283764379673]
plt.scatter(fa_list,accuracy_list)
plt.scatter(fa_list,auc_list)
plt.scatter(fa_list,f1_list)
plt.legend(['Accuracy','ROC AUC score', 'F1 score'])
plt.show()


C:\Users\Pedro\AppData\Local\Temp\ipykernel_8296\1590880250.py:24: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [9]:
score(df_test_y,pred_test)

(0.908095, 0.5705903378676136, 0.24437836578648067)

In [ ]:
Fa=[100,85,75,60,40,20]
accuracy=[0.9045471428571429,0.900395]
auc_score=[0.5378830396854314,0.5031788887870942]
f1_score0=[0.95,0.95,0.95]
f1_score1=[0.24,0.13,0.01 ]



In [53]:
pred_test=model.predict(df_test_x)
print(f'The accuracy on the test set is {accuracy_score(df_test_y,pred_test)} and the AUC is {roc_auc_score(df_test_y,pred_test)}')

The accuracy on the test set is 0.908095 and the AUC is 0.5705903378676136


In [19]:
pred_base=np.zeros_like(pred_test)
print(accuracy_score(df_test_y,pred_base))
print(roc_auc_score(df_test_y,pred_base))

0.9001916666666666
0.5


In [ ]:
from sklearn.ensemble import RandomForestClassifier
rforestmodel=RandomForestClassifier(n_jobs=-1)
rforestmodel.fit(df_train_x,df_train_y)
pred_test=rforestmodel.predict(df_test_x)

In [23]:
print(accuracy_score(df_test_y,pred_test))
print(roc_auc_score(df_test_y,pred_test))

0.9002816666666666
0.504177515095089


In [39]:
import seaborn as sns
for col in data.columns:
    sns.kdeplot(data[data['activity'] == 0][col], label="Pasivas", fill=True)
    sns.kdeplot(data[data['activity'] == 1][col], label="Activas", fill=True)
    plt.title(col)
    plt.legend()
    plt.savefig(f"{col}_distribution.png")  # Guarda la imagen
    plt.close()  # Cierra la figura para liberar memoria

C:\Users\Pedro\AppData\Local\Temp\ipykernel_12060\3059332063.py:3: UserWarning: Dataset has 0 variance; skipping density estimate. Pass `warn_singular=False` to disable this warning.
  sns.kdeplot(data[data['activity'] == 0][col], label="Pasivas", fill=True)
C:\Users\Pedro\AppData\Local\Temp\ipykernel_12060\3059332063.py:4: UserWarning: Dataset has 0 variance; skipping density estimate. Pass `warn_singular=False` to disable this warning.
  sns.kdeplot(data[data['activity'] == 1][col], label="Activas", fill=True)
C:\Users\Pedro\AppData\Local\Temp\ipykernel_12060\3059332063.py:6: UserWarning: No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
  plt.legend()


In [53]:
import joblib
density=0.1
fa=100
model,features,target,df_test_y,df_test_x=joblib.load(f"phia{density}/gb-model-{density}-Fa{fa}.pkl")

In [54]:
pred=model.predict(df_test_x)

In [55]:
accuracy_score(df_test_y,pred)

0.91495

In [56]:
roc_auc_score(df_test_y,pred)

0.6266326880740732

In [16]:
import numpy as np
l1=[1,2,3,4]
l2=[10,20,30,40]
d=dict(zip(l1,l2))

In [17]:
for i in l1:
    print(d[i])

10
20
30
40
